In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.neighbors import NearestNeighbors
import ast
import torch
import clip
from PIL import Image

###### Выгрузка гугл диска в GoogleColab (1 раз)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# *CLIP*
###### Данная модель заменяет собой реализацию кодирования изображения в вектор из предыдущей лабораторной работы (а именно связку SIFT + K-Means + Histogram).



In [ ]:
torch.cuda.is_available()

In [ ]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load('ViT-B/32', device=device)

In [ ]:
def image_to_vector(image, model):
    # Convert image
    image = Image.fromarray(image)
    image = transform(image).unsqueeze(0).to(device)

    # Encode using CLIP
    with torch.no_grad():
        image_features = model.encode_image(image)

    # Normalize
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy()

# Step 5
#### Create indexed DB

In [ ]:
# Create pd.DataFrame for all images in path
def create_database(path, model):
    data = []
    for index, filename in enumerate(os.listdir(path)):
        img = cv2.imread(os.path.join(path, filename))
        if img is not None:
            vector = image_to_vector(img, model)
            if vector is not None:
                data.append({'index': index, 'path': os.path.join(path, filename), 'vector': vector.tolist()[0]})
    df = pd.DataFrame(data)
    return df